In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
import folium
from folium.plugins import HeatMap
import sqlite3

# Carregar dados do banco
def carregar_dados_atualizados(nome_banco, nome_tabela):
    conn = sqlite3.connect(nome_banco)
    try:
        df = pd.read_sql_query(f"SELECT * FROM {nome_tabela}", conn)
    finally:
        conn.close()
    return df

# Função para filtrar e processar os atrasos
def filtrar_e_processar_atrasos(dados):
    df_atrasados = dados.loc[dados['SITUACAO'] == 'ATRASADO'].dropna(subset=['LAT_IN_TIME', 'LON_IN_TIME']).reset_index(drop=True)
    return df_atrasados

# Função para criar o mapa de calor
def criar_mapa_de_calor(df_atrasados):
    latitude_media = df_atrasados['LAT_IN_TIME'].mean()
    longitude_media = df_atrasados['LON_IN_TIME'].mean()
    mapa = folium.Map(location=[latitude_media, longitude_media], zoom_start=13)
    heat_data = df_atrasados[['LAT_IN_TIME', 'LON_IN_TIME']].values.tolist()
    HeatMap(heat_data).add_to(mapa)
    return mapa

# Função principal para atualizar o mapa e salvar no buffer
def atualizar_mapa():
    dados = carregar_dados_atualizados('dados_onibus.db', 'onibus_buffer')
    df_atrasados = filtrar_e_processar_atrasos(dados)
    mapa = criar_mapa_de_calor(df_atrasados)
    mapa.save("mapa_buffer.html")  # Salva o mapa em um arquivo HTML no buffer

# Executa a atualização e salva o mapa
if __name__ == "__main__":
    atualizar_mapa()
